In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from keras.models import Sequential 
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
df = pd.read_csv("./Data/E0_17_18.csv")

In [4]:
df.shape

(380, 65)

In [5]:
df.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,E0,11/08/17,Arsenal,Leicester,4,3,H,2,2,D,...,2.32,21,-1.00,1.91,1.85,2.10,2.02,1.49,4.73,7.25
1,E0,12/08/17,Brighton,Man City,0,2,A,0,0,D,...,2.27,20,1.50,1.95,1.91,2.01,1.96,11.75,6.15,1.29
2,E0,12/08/17,Chelsea,Burnley,2,3,A,0,3,A,...,2.23,20,-1.75,2.03,1.97,1.95,1.90,1.33,5.40,12.25
3,E0,12/08/17,Crystal Palace,Huddersfield,0,3,A,0,2,A,...,1.72,18,-0.75,2.10,2.05,1.86,1.83,1.79,3.56,5.51
4,E0,12/08/17,Everton,Stoke,1,0,H,1,0,H,...,1.76,19,-0.75,1.94,1.90,2.01,1.98,1.82,3.49,5.42


In [11]:
# iloc gets rows (or columns) at particular positions in the index (so it only takes integers).
result_16 = df.iloc[:,:23]
# Axis 0 will act on all the ROWS in each COLUMN
# Axis 1 will act on all the COLUMNS in each ROW
result_16 = result_16.drop(['Div','Date'],axis=1)

# 7 is FTR - so selects as far over as FTR
table_features = df.iloc[:,:7]
table_features = table_features.drop(['FTHG','FTAG','Div','Date'],axis=1)
# All beating columns
bet_16 = df.iloc[:,23:]

In [12]:
result_16.head()
feature_table = df.iloc[:,:23]

In [13]:
# 0) Team, 1) Home Goals Score, 2) Away Goals Score, 3) Home Attack Strength 
# 4) Away Attack Strength 5) Home Goals Conceded, 6) Away Goals Conceded, 
# 7) Home Defensive Strenght, 8) Away Defensive Strength
# New table of columns created
table_16 = pd.DataFrame(columns=('Team','HGS','AGS','HAS','AAS','HGC','AGC','HDS','ADS',))
print(table_16)

Empty DataFrame
Columns: [Team, HGS, AGS, HAS, AAS, HGC, AGC, HDS, ADS]
Index: []


In [22]:
# Full Time Home Goals total divided by / number of rows
avg_home_scored_16 = result_16.FTHG.sum()*1.0 / result_16.shape[0]
# Full Time Away Goals total divided by / number of rows
avg_away_scored_16 = result_16.FTAG.sum()*1.0 / result_16.shape[0]
#
avg_home_conceded_16 = avg_away_scored_16
#
avg_away_conceded_16 = avg_home_scored_16
print("Average number of goals at home",avg_home_scored_16)
print("Average number of goals away", avg_away_scored_16)
print("Average number of goals conceded at home",avg_home_conceded_16)
print("Average number of goals conceded away",avg_away_conceded_16)

Average number of goals at home 1.5315789473684212
Average number of goals away 1.1473684210526316
Average number of goals conceded at home 1.1473684210526316
Average number of goals conceded away 1.5315789473684212


In [23]:
# Grouping HomeTeam and AwayTeam
# Result home - is grouped by 'HomeTeam' and declared an index of the new table: Result Home table.
# Result Away - is grouped by 'AwayTeam' and declared an index of the new table: Result Away table.

result_home = result_16.groupby('HomeTeam', as_index=True)
result_away = result_16.groupby('AwayTeam', as_index=True)

In [24]:
#list(result_home)[0]
result_16.columns

Index(['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR',
       'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY',
       'HR', 'AR'],
      dtype='object')

In [25]:
result_home = result_16.groupby('HomeTeam', as_index=True).agg(np.sum)
result_home

,FTHG,FTAG,HTHG,HTAG,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
HomeTeam,,,,,,,,,,,,,,,,
Arsenal,54,20,23,10,341,184,146,69,186,195,130,93,25,23,1,2
Bournemouth,26,30,8,11,267,256,82,85,165,207,115,84,31,35,1,1
Brighton,24,25,8,11,217,243,71,87,198,152,90,98,37,16,1,1
Burnley,16,17,7,5,207,263,66,80,167,227,94,110,27,16,0,4
Chelsea,30,16,15,6,356,168,132,51,183,222,148,70,19,33,3,1
Crystal Palace,29,27,14,16,267,206,91,81,215,215,104,96,38,36,0,2
Everton,28,22,7,9,189,241,60,81,234,196,81,99,28,23,1,2
Huddersfield,16,25,10,12,207,186,55,77,180,180,102,83,26,32,2,1
Leicester,25,22,11,11,227,216,75,69,180,195,112,89,22,28,2,1


In [26]:
result_away = result_16.groupby('AwayTeam', as_index=True).agg(np.sum)
result_away

,FTHG,FTAG,HTHG,HTAG,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
AwayTeam,,,,,,,,,,,,,,,,
Arsenal,31,20,11,7,238,250,79,87,194,197,86,93,25,32,2,1
Bournemouth,31,19,14,9,292,199,91,75,232,172,126,102,37,24,0,0
Brighton,29,10,13,8,312,166,94,47,141,215,130,73,13,17,1,1
Burnley,22,20,7,9,299,172,84,61,214,183,130,73,27,38,2,0
Chelsea,22,32,7,15,209,250,69,88,195,180,89,82,40,22,0,1
Crystal Palace,28,16,13,4,256,208,95,60,199,210,113,105,34,33,1,0
Everton,36,16,16,8,301,171,102,62,187,225,113,69,23,23,3,2
Huddersfield,33,12,7,5,255,153,88,56,197,216,126,63,23,35,3,1
Leicester,38,31,15,15,274,195,99,73,172,185,127,91,24,30,0,3


In [27]:
# Home Team  
table_16.Team = result_home.index.values
# Home Goals Score
table_16.HGS = result_home.FTHG.values
# Home Goals Conceded 
table_16.HGC = result_home.FTAG.values
# Attack Goals Strength
table_16.AGS = result_away.FTAG.values
# Away Goals Conceded
table_16.AGC = result_away.FTHG.values
# Home Attack Strength
# table_16.HAS = result_away.FTHG.values
# Display
table_16

,Team,HGS,AGS,HAS,AAS,HGC,AGC,HDS,ADS
0,Arsenal,54,20,NaN,NaN,20,31,NaN,NaN
1,Bournemouth,26,19,NaN,NaN,30,31,NaN,NaN
2,Brighton,24,10,NaN,NaN,25,29,NaN,NaN
3,Burnley,16,20,NaN,NaN,17,22,NaN,NaN
4,Chelsea,30,32,NaN,NaN,16,22,NaN,NaN
5,Crystal Palace,29,16,NaN,NaN,27,28,NaN,NaN
6,Everton,28,16,NaN,NaN,22,36,NaN,NaN
7,Huddersfield,16,12,NaN,NaN,25,33,NaN,NaN
8,Leicester,25,31,NaN,NaN,22,38,NaN,NaN
9,Liverpool,45,39,NaN,NaN,10,28,NaN,NaN


In [28]:
#Assuming number of home games = number of away games
num_games = result_16.shape[0]/20

In [29]:
table_16.HAS = (table_16.HGS / num_games) / avg_home_scored_16
table_16.AAS = (table_16.AGS / num_games) / avg_away_scored_16
table_16.HDS = (table_16.HGC / num_games) / avg_home_conceded_16
table_16.ADS = (table_16.AGC / num_games) / avg_away_conceded_16
table_16

,Team,HGS,AGS,HAS,AAS,HGC,AGC,HDS,ADS
0,Arsenal,54,20,1.855670,0.917431,20,31,0.917431,1.065292
1,Bournemouth,26,19,0.893471,0.871560,30,31,1.376147,1.065292
2,Brighton,24,10,0.824742,0.458716,25,29,1.146789,0.996564
3,Burnley,16,20,0.549828,0.917431,17,22,0.779817,0.756014
4,Chelsea,30,32,1.030928,1.467890,16,22,0.733945,0.756014
5,Crystal Palace,29,16,0.996564,0.733945,27,28,1.238532,0.962199
6,Everton,28,16,0.962199,0.733945,22,36,1.009174,1.237113
7,Huddersfield,16,12,0.549828,0.550459,25,33,1.146789,1.134021
8,Leicester,25,31,0.859107,1.422018,22,38,1.009174,1.305842
9,Liverpool,45,39,1.546392,1.788991,10,28,0.458716,0.962199


In [30]:
table_16[table_16.Team == "Everton"]

,Team,HGS,AGS,HAS,AAS,HGC,AGC,HDS,ADS
6,Everton,28,16,0.962199,0.733945,22,36,1.009174,1.237113


In [31]:
# Building a feature dataframe
# So getting data and appending to the end of the dataframe/ feature table

''' feature_table contains all the fixtures in the season.
ftr = full time result
hst = home shots on target
ast = away shots on target
'''
feature_table = feature_table[['HomeTeam','AwayTeam','FTR','HST','AST']]

f_HAS = []
f_HDS = []
f_AAS = []
f_ADS = []
for index,row in feature_table.iterrows():
    f_HAS.append(table_16[table_16['Team'] == row['HomeTeam']]['HAS'].values[0])
    f_HDS.append(table_16[table_16['Team'] == row['HomeTeam']]['HDS'].values[0])
    f_AAS.append(table_16[table_16['Team'] == row['AwayTeam']]['AAS'].values[0])
    f_ADS.append(table_16[table_16['Team'] == row['AwayTeam']]['ADS'].values[0])
    
feature_table['HAS'] = f_HAS
feature_table['HDS'] = f_HDS
feature_table['AAS'] = f_AAS
feature_table['ADS'] = f_ADS

In [32]:
feature_table.head(26)

,HomeTeam,AwayTeam,FTR,HST,AST,HAS,HDS,AAS,ADS
0,Arsenal,Leicester,H,10,3,1.855670,0.917431,1.422018,1.305842
1,Brighton,Man City,A,2,4,0.824742,1.146789,2.064220,0.446735
2,Chelsea,Burnley,A,6,5,1.030928,0.733945,0.917431,0.756014
3,Crystal Palace,Huddersfield,A,4,6,0.996564,1.238532,0.550459,1.134021
4,Everton,Stoke,H,4,1,0.962199,1.009174,0.688073,1.305842
5,Southampton,Swansea,D,2,0,0.687285,1.192661,0.504587,1.099656
6,Watford,Liverpool,D,4,5,0.927835,1.422018,1.788991,0.962199
7,West Brom,Bournemouth,H,6,2,0.721649,1.330275,0.871560,1.065292
8,Man United,West Ham,H,6,1,1.305842,0.412844,1.100917,1.443299
9,Newcastle,Tottenham,A,3,6,0.721649,0.779817,1.559633,0.687285


In [161]:
def transformResult(row):
    '''Converts results (H,A or D) into numeric values'''
    if(row.FTR == 'H'):
        return 1
    elif(row.FTR == 'A'):
        return 0
#     else:
#         return 0

In [162]:
# Converting from H, D, A into numeric value
# lamba more powerful way - for each row in table
feature_table["Result"] = feature_table.apply(lambda row: transformResult(row),axis=1)

In [163]:
# Season 2016-2017 / Season 2017-2018
# split data into X and y
X_train = feature_table[['HST','AST','HAS','HDS','AAS','ADS',]][:370]#
y_train = feature_table['Result'][:370]
# X_train = feature_table[['HST','AST','HAS','HDS','AAS','ADS',]][:300]#
# y_train = feature_table['Result'][:300]

X_test = feature_table[['HST','AST','HAS','HDS','AAS','ADS',]][270: 370]# 
y_test = feature_table['Result'][270:370]
# X_test = feature_table[['HST','AST','HAS','HDS','AAS','ADS',]][300: 370]# 
# y_test = feature_table['Result'][300:370]

X_predict = feature_table[['HST','AST','HAS','HDS','AAS','ADS',]][370:]# 
y_predict = feature_table['Result'][370:]

In [164]:
# Season 2016-2017 / Season 2017-2018
# split data into X and y
X_train = feature_table[['HST','AST','HAS','HDS','AAS','ADS']]#
y_train = feature_table['Result']
feature_table.head()

,HomeTeam,AwayTeam,FTR,HST,AST,HAS,HDS,AAS,ADS,Result
0,Arsenal,Leicester,H,10,3,1.855670,0.917431,1.422018,1.305842,1.0
1,Brighton,Man City,A,2,4,0.824742,1.146789,2.064220,0.446735,0.0
2,Chelsea,Burnley,A,6,5,1.030928,0.733945,0.917431,0.756014,0.0
3,Crystal Palace,Huddersfield,A,4,6,0.996564,1.238532,0.550459,1.134021,0.0
4,Everton,Stoke,H,4,1,0.962199,1.009174,0.688073,1.305842,1.0


In [165]:
game_for_prediction = feature_table.loc[feature_table['HomeTeam'] == 'Chelsea']
game_for_prediction = game_for_prediction.loc[game_for_prediction['AwayTeam'] == 'Burnley']
game_for_prediction

,HomeTeam,AwayTeam,FTR,HST,AST,HAS,HDS,AAS,ADS,Result
2,Chelsea,Burnley,A,6,5,1.030928,0.733945,0.917431,0.756014,0.0


In [166]:
feature_names = feature_table[(feature_table['HomeTeam'] == 'Chelsea') & (feature_table['AwayTeam'] == 'Burnley')].index

In [167]:
feature_table_new = feature_table.drop(feature_names)

In [168]:
feature_table_new.head()

,HomeTeam,AwayTeam,FTR,HST,AST,HAS,HDS,AAS,ADS,Result
0,Arsenal,Leicester,H,10,3,1.855670,0.917431,1.422018,1.305842,1.0
1,Brighton,Man City,A,2,4,0.824742,1.146789,2.064220,0.446735,0.0
3,Crystal Palace,Huddersfield,A,4,6,0.996564,1.238532,0.550459,1.134021,0.0
4,Everton,Stoke,H,4,1,0.962199,1.009174,0.688073,1.305842,1.0
5,Southampton,Swansea,D,2,0,0.687285,1.192661,0.504587,1.099656,NaN


In [169]:
X = feature_table_new[['HST','AST','HAS','HDS','AAS','ADS']]
y = feature_table_new['Result']
#for prediction
Xnew = game_for_prediction[['HST','AST','HAS','HDS','AAS','ADS']]
yreal = game_for_prediction['Result']

In [177]:
model = Sequential()
model.add(Dense(5, input_dim=6, kernel_initializer='normal', activation='softmax'))
model.add(Dense(1, kernel_initializer='normal'))
# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

ValueError: The model cannot be compiled because it has no loss to optimize.

In [176]:
history = model.fit(X, y, validation_split=0.20, epochs=10, batch_size=20,verbose=1)
# Plot training and validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train','Test'], loc='upper left')
plt.show()

Train on 303 samples, validate on 76 samples
Epoch 1/10


InvalidArgumentError: Received a label value of -9223372036854775808 which is outside the valid range of [0, 1).  Label values: 1 1 1 0 -9223372036854775808 1 1 1 0 1 0 -9223372036854775808 0 1 0 1 1 0 0 1
	 [[{{node loss_15/dense_41_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits}} = SparseSoftmaxCrossEntropyWithLogits[T=DT_FLOAT, Tlabels=DT_INT64, _class=["loc:@train...s_grad/mul"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](loss_15/dense_41_loss/Log, loss_15/dense_41_loss/Cast)]]

In [93]:
ynew = model.predict(Xnew)
ynew

array([[0.18364355]], dtype=float32)